In [1]:
# Mount Google Drive to save generated audio files
from google.colab import drive
drive.mount('/content/drive')
drive_path = '/content/drive/MyDrive/CART498/'

Mounted at /content/drive


In [ ]:
# Install dependencies for Stable Audio inference
!pip -q install diffusers transformers accelerate torchsde soundfile

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 4.3 MB/s eta 0:00:00


In [ ]:
# Hugging Face login
from huggingface_hub import login
try:
    from google.colab import userdata
    token = userdata.get('HF_TOKEN')
    if token:
        login(token=token)
    else:
        login()
except Exception:
    login()

## Generate sounds


In [ ]:
# Load Stable Audio model and generate variations via different seeds
import torch
import soundfile as sf
from diffusers import StableAudioPipeline
from IPython.display import Audio

pipe = StableAudioPipeline.from_pretrained(
    "stabilityai/stable-audio-open-1.0",
    torch_dtype=torch.float16,
)
pipe = pipe.to("cuda")

# single prompt, many variations
prompt = "Fictional animal vocalizations in a magical forest."
negative_prompt = "Low quality."

num_outputs = 15

generator = torch.Generator("cuda")

for i in range(num_outputs):
    # different seed each time for variation
    generator.manual_seed(1000 + i)
    audio = pipe(
        prompt,
        negative_prompt=negative_prompt,
        num_inference_steps=200,
        audio_end_in_s=21.0,
        num_waveforms_per_prompt=1,
        generator=generator,
    ).audios

    output = audio[0].T.float().cpu().numpy()
    filename = f"{drive_path}animal_sound_{i:02d}.wav"
    sf.write(filename, output, pipe.vae.sampling_rate)


Output hidden; open in https://colab.research.google.com to view.